<a href="https://colab.research.google.com/github/Zain12561/Customer-Churn-Analysis/blob/main/Ckplus48.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
!pip install tensorflow

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
import os
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils import class_weight
from google.colab import files


In [4]:
from google.colab import files
uploaded = files.upload()  # select CK+48.zip

Saving CK+48.zip to CK+48.zip


In [58]:
!rm -rf ./ckplus/CK+48
!unzip CK+48.zip -d ./ckplus


Archive:  CK+48.zip
   creating: ./ckplus/CK+48/
   creating: ./ckplus/CK+48/anger/
 extracting: ./ckplus/CK+48/anger/S010_004_00000017.png  
 extracting: ./ckplus/CK+48/anger/S010_004_00000018.png  
 extracting: ./ckplus/CK+48/anger/S010_004_00000019.png  
 extracting: ./ckplus/CK+48/anger/S011_004_00000019.png  
 extracting: ./ckplus/CK+48/anger/S011_004_00000020.png  
 extracting: ./ckplus/CK+48/anger/S011_004_00000021.png  
 extracting: ./ckplus/CK+48/anger/S014_003_00000028.png  
 extracting: ./ckplus/CK+48/anger/S014_003_00000029.png  
 extracting: ./ckplus/CK+48/anger/S014_003_00000030.png  
 extracting: ./ckplus/CK+48/anger/S022_005_00000030.png  
 extracting: ./ckplus/CK+48/anger/S022_005_00000031.png  
 extracting: ./ckplus/CK+48/anger/S022_005_00000032.png  
 extracting: ./ckplus/CK+48/anger/S026_003_00000013.png  
 extracting: ./ckplus/CK+48/anger/S026_003_00000014.png  
 extracting: ./ckplus/CK+48/anger/S026_003_00000015.png  
 extracting: ./ckplus/CK+48/anger/S028_001_000

In [64]:
dataset_path = './ckplus/CK+48'
print(os.listdir(dataset_path))  # should show all 7 class folders


['fear', 'surprise', 'sadness', 'disgust', 'anger', 'happy', 'contempt']


In [85]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(48,48),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(48,48),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)


Found 788 images belonging to 7 classes.
Found 193 images belonging to 7 classes.


In [65]:
for cls in os.listdir(dataset_path):
    path = os.path.join(dataset_path, cls)
    print(cls, len(os.listdir(path)))


fear 75
surprise 249
sadness 84
disgust 177
anger 135
happy 207
contempt 54


In [86]:
train_labels = train_generator.classes
class_weights_array = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = dict(enumerate(class_weights_array))
print("Class weights:", class_weights)


Class weights: {0: np.float64(1.0423280423280423), 1: np.float64(2.5584415584415585), 2: np.float64(0.7927565392354124), 3: np.float64(1.8761904761904762), 4: np.float64(0.6781411359724613), 5: np.float64(1.6554621848739495), 6: np.float64(0.5628571428571428)}


In [87]:
num_classes = train_generator.num_classes

model = Sequential([
    Input(shape=(48,48,1)),
    Conv2D(32, (3,3), activation='relu', padding='same'),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu', padding='same'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu', padding='same'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)              │ (None, 48, 48, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 24, 24, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 24, 24, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_20 (MaxPooling2D) │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 256)            │     1,179,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,274,375 (4.86 MB)

 Trainable params: 1,274,375 (4.86 MB)

 Non-trainable params: 0 (0.00 B)

In [88]:
early_stop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    class_weight=class_weights,  # optional
    callbacks=[early_stop]
)


Epoch 1/50


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


25/25 ━━━━━━━━━━━━━━━━━━━━ 11s 290ms/step - accuracy: 0.1937 - loss: 1.9463 - val_accuracy: 0.2798 - val_loss: 1.9237
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 175ms/step - accuracy: 0.2861 - loss: 1.9063 - val_accuracy: 0.4093 - val_loss: 1.7250
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 204ms/step - accuracy: 0.4364 - loss: 1.7085 - val_accuracy: 0.4974 - val_loss: 1.3592
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 204ms/step - accuracy: 0.5759 - loss: 1.3313 - val_accuracy: 0.6373 - val_loss: 1.0720
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 195ms/step - accuracy: 0.6864 - loss: 0.9505 - val_accuracy: 0.7824 - val_loss: 0.6655
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 235ms/step - accuracy: 0.7945 - loss: 0.6589 - val_accuracy: 0.8135 - val_loss: 0.6189
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 183ms/step - accuracy: 0.8448 - loss: 0.4995 - val_accuracy: 0.8187 - val_loss: 0.5783
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 199ms/step - accuracy: 0.8480 - loss: 0.4716 - val_accuracy: 0.7720 - val

In [89]:
print("Overall Accuracy:", accuracy*100, "%")
print(classification_report(y_true, y_pred, target_names=list(val_generator.class_indices.keys())))


Overall Accuracy: 13.471502590673575 %
              precision    recall  f1-score   support

       anger       0.10      0.15      0.12        27
    contempt       0.00      0.00      0.00        10
     disgust       0.08      0.06      0.07        35
        fear       0.04      0.07      0.05        15
       happy       0.22      0.20      0.21        41
     sadness       0.12      0.12      0.12        16
    surprise       0.21      0.18      0.20        49

    accuracy                           0.13       193
   macro avg       0.11      0.11      0.11       193
weighted avg       0.14      0.13      0.14       193



In [90]:
val_generator.reset()
preds = model.predict(val_generator)
y_pred = np.argmax(preds, axis=1)
y_true = val_generator.classes

accuracy = accuracy_score(y_true, y_pred)
print("Overall Accuracy:", accuracy*100, "%")

print(classification_report(
    y_true, y_pred, target_names=list(val_generator.class_indices.keys())
))


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step
Overall Accuracy: 85.49222797927462 %
              precision    recall  f1-score   support

       anger       0.69      0.67      0.68        27
    contempt       1.00      0.70      0.82        10
     disgust       0.93      0.77      0.84        35
        fear       0.67      0.80      0.73        15
       happy       0.95      1.00      0.98        41
     sadness       0.61      0.69      0.65        16
    surprise       0.94      1.00      0.97        49

    accuracy                           0.85       193
   macro avg       0.83      0.80      0.81       193
weighted avg       0.86      0.85      0.85       193



In [91]:
print("Overall Accuracy:", accuracy*100, "%")


Overall Accuracy: 85.49222797927462 %
